# Sketchem: A Molecular Pictionary 
---

This notebook is designed to provide an interactive and visual introduction to Sketchem and its underlying codebase. Our objective is to help you get started with Sketchem and understand its core features without diving too much into implementation details. Each section contains explanations, code demonstrations, and visualizations to help you understand its capabilities. Feel free to run the code cells to see Sketchem in action.

Let's dive in!


## 1. Introduction: The Chemistry of Molecular Representation

### 1.1 Context

One of chemistry's pillars is the ability to draw and recognize molecules. This is a skill that is taught to chemistry students from the very first class. It's how chemists communicate with each other and present their findings. An important part of any chemist's career is learning the ins and outs of chemical structures and to be able to draw them.

To many, however, this skill can be intimidating at first. Worse, by the third time you learn nomenclature and the rules only seem to get more complex, it can become boring and demotivating. And once you think you got the hang of it, you have to start memorising amino acids and other classes of molecules... Traditional tools like flashcards and textbooks are not very effective in teaching this skill. They don't provide any feedback and often lead to frustration, which reduces learning efficiency. 

### 1.2 Our idea

Sketchem was largely inspired by a class we had to take last semester: Biochemistry. Every student in the class had to learn the same amino acids by heart. As we spent hours tediously memorising each molecule, we thought: “There must be a better way of learning this.”


And we did find it: Whiteboards. Whiteboards were nice, except everyone had to bring their own pen and often there simply wasn't enough room on one whiteboard for everyone to draw 20 amino acids. 


That’s when the idea for Sketchem came to life. A multiplayer molecular pictionary game. But not just your typical molecule drawing game using your typical bond / atom molecule structure editor. We wanted it to *feel* like a whiteboard. Hence, our idea evolved into something even more complex: a drawing gamethat can identify hand-drawn molecular structures and convert them into standardized SMILES notation, and then give you instantaneous feedback on whether you drew the correct molecule or not.


As we started designing it, we realized that there were countless fields of chemistry that could benefit from this tool. Thus, there was no way the three of us could write down every category students would want to learn. Hence, an additional idea for an AI category generator was born. 


### 1.3 The Challenge of Chemical Structure Recognition

Chemical structures are the visual language of chemistry. Chemists communicate molecular structures through standardized drawings that represent atoms, bonds, and spatial arrangements. However, translating between hand-drawn structures and computer-readable formats presents significant challenges:

1. **Variability in Drawing Styles**: The same molecule can look very different when drawn by different people. Factors like varying bond angles, atom placements, or even shorthand notations can confuse recognition systems.

2. **Complexity of Representation**: Chemical structures can range from simple linear molecules to complex polycyclic compounds with multiple functional groups.

3. **Standardization Requirements**: Before a computer can "understand" a structure, it must be translated into a standard format like SMILES.

### 1.4  SMILES Notation: The Digital Language of Chemistry

SMILES (Simplified Molecular Input Line Entry System) is a way to represent chemical structures using a line of text. It captures atoms, bonds, branching, and rings with a consistent syntax, making molecules easy to store, compare, and search computationally.

- Examples:
  - Methane → C
  - Ethanol → CCO
  - Benzene → c1ccccc1

SMILES makes it possible for chemical software, databases (like PubChem), and AI models to work with molecules.





## 2.Material & Methods


### 2.1 Architecture Overview

Sketchem is built using a modern web application stack:

- **Frontend**: Streamlit for the user interface and interactive elements.

- **Drawing Interface**: Streamlit-drawable-canvas for the molecular drawing canvas.

- **AI Integration**: Google Gemini analyzes the sketches and predicts the most likely molecule name based on visual input; Gemini is also used for creating categories of molecules based on user prompts.

- **Chemical Processing**: RDKit for molecular structure comparison and validation.

- **Check and provide feedback**: The molecule name provided by Google Gemini is transformed into a SMILES string and compared to PubChem’s database of known molecules, and the structure is validated or not. Sketchem can then provide feedback to the user based on the correctness of the drawn molecule.

- **State Management**: Handled with Streamlit’s session state, ensures game progress and user inputs are tracked.

- **Multiplayer**: In-memory database for the multiplayer functionality.



### 2.2 Setting Up Our Environment

Now that we've gotten through all the technical stuff, let's test these awesome features, shall we? 

But first, let's import all the necessary dependencies. Sketchem relies on several key libraries for all of its awesome features!

In [ ]:
from sketchem.utils.smiles_validator_ai import validate_drawing_with_ai
from rdkit import Chem
from rdkit.Chem import Draw
import os
from sketchem.utils.create_category import get_molecules_for_category_pubchem
from PIL import Image
from sketchem.utils.environment import get_gemini_api_key
from sketchem.utils.smiles_validator_ai import get_molecule_with_ai
import matplotlib.pyplot as plt
%matplotlib inline


### 2.3 AI-Powered Category Creation

Our web app lets you practice molecules not only from default categories, but also from **custom lists generated by AI**. This means you can tailor the game to match your courses, study goals, or curiosity.


Here’s how it works:

1. Click "Create a molecule category using AI"

2. Type in the category you want to create

2. Click “Generate Category” and wait for AI to generate the category

3. Click “Submit” to confirm the category

And that’s it! Your custom category is created, and you’re ready to go.


Try out a demo of the category creation function below, and see for yourself: 

In [ ]:

# Get API key -> make sure you have one (see readme)
api_key = get_gemini_api_key()

# Example prompts to try
example_prompts = [
    "Common amino acids",
    "Simple aromatic compounds",
    "Molecules used in pain medication"
    # Add more prompts here
]

# Select one prompt to demonstrate
selected_prompt = example_prompts[0] # Change this to try different prompts
print(f"Creating category with prompt: '{selected_prompt}'")

# Create the category
result = get_molecules_for_category_pubchem(api_key=api_key, user_prompt=selected_prompt, jupyternb=True)
print(f"Final Result after PubChem API calls: {result}")


 ### 2.4 AI-Powered Molecule Recognition: The Heart of Sketchem

At the core of Sketchem is its AI-powered **recognition** system, which uses Google's Gemini AI to interpret hand-drawn molecular structures. The AI has been specifically prompted to understand chemical notation and convert drawings to common molecule names.

The overall process of Sketchem is:

Drawing → Image → Bytes → Gemini AI → Names → SMILES String → Validation → Feedback

AI-powered recognition focuses on the 'Drawing → SMILES' portion 

#### The Recognition Process

1. **Image Capture**: The drawing is captured from the canvas

2. **Preprocessing**: The image is processed to transform it into an easy-to-share format: bytes

3. **AI Analysis**: The image bytes are sent to Gemini AI with the specialized prompt

4. **Name Generation**: The AI generates 3 common molecular names (synonyms) based on the drawing. If it cannot find 3 different names, it output the same name 3 times, or two names and one of the two repeated.

5. **SMILES Conversion**: The generated names are converted to SMILES strings using the PubChem API. If none of the three names are not found in PubChem, the app throws an error in the form of a toast message, and asks the user to redraw the molecule. If at least one name is found, the app uses its SMILES string for validation.


#### Challenges in Molecular Recognition

Recognizing hand-drawn chemical structures presents several challenges:

1. **Variability in Drawing Styles**: Different users draw the same molecule differently

2. **Ambiguity in Representation**: There exists multiple ways to represent the same molecule: skeletal, condensed, etc.

The system must be strict enough to ensure chemical accuracy but flexible enough to accommodate different drawing styles and structural representations. Sketchem addresses these challenges through its specialized AI prompt and validation system, which combines the flexibility of AI interpretation with the strictness of chemical validation. This combination between AI and chemistry ensures feedback is both fast and accurate. 

Try out it out for yourslef in the code below!

In [ ]:
# For demonstration purposes, we'll use some pre-defined images from the data folder and SMILES strings 

sample_molecules = {
    "Acetic acid": "CC(=O)O",
    "Adenine": "Nc1ncnc2[nH]cnc12",
    "Cyclohexane": "C1CCCCC1", 
} 

# Path to test data (images)
TEST_DATA_DIR = "../data/testdata"
api_key = get_gemini_api_key()

for name, smiles in sample_molecules.items():
    print(f"## Testing {name} (Expected SMILES: {smiles})")
    
    # Find good and bad images for this molecule
    good_image_path = os.path.join(TEST_DATA_DIR, f"{name.lower().replace(' ', '_')}_good.jpg")
    bad_image_path = os.path.join(TEST_DATA_DIR, f"{name.lower().replace(' ', '_')}_bad.jpg")
    
    for img_path, quality in [(good_image_path, "Good"), (bad_image_path, "Bad")]:
        if os.path.exists(img_path):
            # Display the test image
            img = Image.open(img_path)
            plt.figure(figsize=(4, 4))
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"{name} ({quality} Drawing)")
            plt.tight_layout()
            plt.show()
            
            # Get SMILES from image using Gemini
            with open(img_path, 'rb') as f:
                image_bytes = f.read()
            
            # Get detected molecule -> results will vary
            detected_response = get_molecule_with_ai(api_key, image_bytes, smiles, jupyternb=True)
            
            # Display the detected molecule names
            if detected_response and hasattr(detected_response, 'text'):
                detected_names = detected_response.text.strip().split('\n')
                names_text = detected_response.text.replace("\n", ", ")
                print(f"Detected names: {names_text}")
                
                
                # Check if the drawing is accepted
                is_accepted = validate_drawing_with_ai(detected_response, smiles, 0.85, jupyternb=True)
                print(f"Is it accepted? {'Yes' if is_accepted else 'No'}")
                
                # If at least one name was found, try to display the molecules side by side
                if detected_names and detected_names[0].strip() and detected_names[0] != "INVALID_STRUCTURE":
                    try:
                        import pubchempy as pcp
                        
                        # Get SMILES for the first detected name
                        compounds = pcp.get_compounds(detected_names[0], 'name')
                        if compounds:
                            detected_smiles = compounds[0].canonical_smiles
                            
                            # Create a figure with two subplots side by side
                            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
                            
                            # Display detected molecule
                            mol1 = Chem.MolFromSmiles(detected_smiles)
                            img1 = Draw.MolToImage(mol1, size=(300, 300))
                            ax1.imshow(img1)
                            ax1.axis('off')
                            ax1.set_title(f"Detected: {detected_names[0]}\n{detected_smiles}")
                            
                            # Display correct molecule
                            mol2 = Chem.MolFromSmiles(smiles)
                            img2 = Draw.MolToImage(mol2, size=(300, 300))
                            ax2.imshow(img2)
                            ax2.axis('off')
                            ax2.set_title(f"Correct molecule: {name}\n{smiles}")
                            
                            plt.tight_layout()
                            plt.show()
                        else:
                            print(f"No compound found for {detected_names[0]}")
                    except Exception as e:
                        print(f"Error displaying molecules: {e}")
                else:
                    print("Invalid structure detected or no names returned")
            else:
                print("Invalid response from Gemini API")
            print("-" * 80)

### 2.4 SMILES Verification


Now let's focus on what happens after the translation has taken place, in other words: SMILES String → Feedback. Once a molecule is drawn and interpreted by the AI, Sketchem performs **SMILES verification** to ensure that the AI’s guess is both **chemically valid** and **correct**.


#### What factors have to be checked?

If we recall, SMILES is a way of translating chemistry into text (Ethanol → `CCO`). There are two factors that have to be checked in the SMILES:

1. Represents a **real, chemically valid molecule**, for instance in the example above, the image of a circle is clearly not a molecule, Sketchem does not recognize this as a SMILES and thus provides appropriate feedback.

2. Matches the **intended target molecule** the user was supposed to draw. For instance if the user draws ethanol instead of methanol, Sketchem converts it into the SMILES notation and is able to compare it to the expected SMILES. Depending on whether they match or not it accepts or rejects the answer.


#### How Sketchem Verifies SMILES

After Gemini AI generates a SMILES string from a drawing:

1. **Validation**:  
   The SMILES is passed to a chemical toolkit (RDKit) to ensure it is syntactically correct and chemically meaningful.

2. **Comparison**:  
   The generated SMILES is compared to the target SMILES using MCS, a technique that finds the largest common substructure between two molecules.
   If the structure matches above a certain threshold, the drawing is considered correct. 


   If the user checks the "Hint" option in the setup, Sketchem displays the AI's guess of the molecule name as feedback. This is especially useful when the guess is close but incorrect. Say, for instance, the correct answer was methanol, but the user draws ethanol. Instead of just saying “Wrong!”, SketChem can say: "What you drew looks more like ethanol."
   This is much more helpful for learning. This also accomodates for possible errors in the AI detection and allows the user to redraw the structure more clearly if needed.



In [ ]:
# Create a mock response object to simulate Gemini API's response style
class MockResponse:
    def __init__(self, text):
        self.text = text

# Function to display molecules
def display_molecule(smiles, title=None): 
    """Display a molecule from SMILES string"""
    mol = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(mol, size=(300, 300))
    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.axis('off')
    plt.title(title or smiles)
    plt.tight_layout()
    plt.show()

# Function to display molecules from names using PubChem
def display_molecules_from_names(names):
    """Display molecules from a list of names using PubChem, showing each unique structure once"""
    import pubchempy as pcp
    
    # Dictionary to store unique SMILES and their corresponding names
    unique_structures = {}
    
    # First pass: collect all unique SMILES and their names
    for name in names:
        try:
            compounds = pcp.get_compounds(name, 'name')
            if compounds:
                smiles = compounds[0].canonical_smiles
                if smiles in unique_structures:
                    unique_structures[smiles].append(name)
                else:
                    unique_structures[smiles] = [name]
            else:
                print(f"No compounds found for {name}")
        except Exception as e:
            print(f"Error retrieving {name}: {e}")
    
    # Second pass: display each unique structure once with all its names
    for smiles, structure_names in unique_structures.items():
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                img = Draw.MolToImage(mol, size=(300, 300))
                plt.figure(figsize=(4, 4))
                plt.imshow(img)
                plt.axis('off')
                names_str = ", ".join(structure_names)
                plt.title(f"{names_str}\n{smiles}")
                plt.tight_layout()
                plt.show()
            else:
                print(f"Could not create molecule from SMILES: {smiles}")
        except Exception as e:
            print(f"Error displaying structure for {smiles}: {e}")

######################################################################################################

# Example target SMILES
target_smiles = "CC(=O)C1=CC=CC=C1"  # Acetophenone

# Create mock responses with molecule names instead of SMILES
exact_match_response = MockResponse("Acetophenone\nPhenyl methyl ketone\n1-phenylethanone")  # Same molecule, different names
high_similarity_response = MockResponse("4-methylacetophenone\np-methylacetophenone\n1-(4-methylphenyl)ethanone")  # Very similar
medium_similarity_response = MockResponse("Benzoic acid\nBenzenecarboxylic acid\nPhenylformic acid")  # Medium similarity
low_similarity_response = MockResponse("Acetic acid\nEthanoic acid\nMethanecarboxylic acid")  # Low similarity
different_response = MockResponse("Benzene\nBenzol\nCyclohexatriene")  # Structural fragment only
invalid_response = MockResponse("INVALID_STRUCTURE")  # Invalid structure response

# Test with different thresholds
thresholds = [1, 0.90, 0.80, 0.70, 0.60, 0.50]

# Display target molecule
display_molecule(target_smiles, title=f"Acetophenone (Target Smiles): {target_smiles}")

print("Testing validate_drawing_with_ai with different responses and thresholds:")

print("\nExact match tests (different names for same structure):")
print(f"Testing names: {exact_match_response.text}")
# Display molecules for the names
display_molecules_from_names(exact_match_response.text.strip().split('\n'))
for threshold in thresholds:
    result = validate_drawing_with_ai(exact_match_response, target_smiles, threshold, jupyternb=True)
    print(f"Threshold {threshold}: {'✓ Match' if result is True else '✗ No match'}")

print("\nHigh similarity tests (4-methylacetophenone):")
print(f"Testing names: {high_similarity_response.text}")
# Display molecules for the names
display_molecules_from_names(high_similarity_response.text.strip().split('\n'))
for threshold in thresholds:
    result = validate_drawing_with_ai(high_similarity_response, target_smiles, threshold, jupyternb=True)
    print(f"Threshold {threshold}: {'✓ Match' if result is True else '✗ No match'}")

print("\nMedium similarity tests (Benzoic acid):")
print(f"Testing names: {medium_similarity_response.text}")
# Display molecules for the names
display_molecules_from_names(medium_similarity_response.text.strip().split('\n'))
for threshold in thresholds:
    result = validate_drawing_with_ai(medium_similarity_response, target_smiles, threshold, jupyternb=True)
    print(f"Threshold {threshold}: {'✓ Match' if result is True else '✗ No match'}")

print("\nLow similarity tests (Acetic acid):")
print(f"Testing names: {low_similarity_response.text}")
# Display molecules for the names
display_molecules_from_names(low_similarity_response.text.strip().split('\n'))
for threshold in thresholds:
    result = validate_drawing_with_ai(low_similarity_response, target_smiles, threshold, jupyternb=True)
    print(f"Threshold {threshold}: {'✓ Match' if result is True else '✗ No match'}")

print("\nDifferent molecule tests (Benzene):")
print(f"Testing names: {different_response.text}")
# Display molecules for the names
display_molecules_from_names(different_response.text.strip().split('\n'))
for threshold in thresholds:
    result = validate_drawing_with_ai(different_response, target_smiles, threshold, jupyternb=True)
    print(f"Threshold {threshold}: {'✓ Match' if result is True else '✗ No match'}")

print("\nInvalid structure test:") # Should get '✗ No match'
print(f"Testing names: {invalid_response.text}")
result = validate_drawing_with_ai(invalid_response, target_smiles, 0.85, jupyternb=True)
print(f"Result: {'✓ Match' if result is True else '✗ No match'}")

### 2.5 Code Structure Overview

Now that we know how the key features work, let's go over the structure of the backend of the code. Here is a summarised version of the code:

```bash
sketchem/

├── .streamlit/             # Streamlit app configuration and API key storage for Streamlit Cloud deployment
├── assets/                 # Holds images (logos, banners, etc.) and other static files used across the docs & UI
├── data/                   # Sample data for testing & examples in this document
├── src/sketchem/
│   ├── main.py            # Entry point for the web application: includes routing, page rendering
│   ├── data/              # Stores the default molecule categories
│   ├── db/                # Houses the in-memory database for the multiplayer game
│   ├── pages/             # Contains the code for each page of the web application
│   └── utils/             # Utility functions and elements used across the application
├── tests/
│   └── test_smiles_validator.py  # Tests for the SMILES validation
└── README.md              # Project overview, install & quickstart

```
From this we can delve into the `pages` folder in `src`. Each `.py` file in this folder corresponds to a page in the web application. For example, `home_page.py` contains the code for the home page of the web application. The style folder also contains the CSS files for the web application, which are responsible for the styling of the web application. 

Another notable folder is the `utils` folder, which contains utility functions that are used across the web application. Here is a small table of each function and what it does: 

| File                       | What it does                                         |
|----------------------------|------------------------------------------------------|
| `smiles_validator_ai.py`   | SMILES validator, which is used to compare the molecules that are generated by the AI to the target SMILES string. |
| `create_category.py`       | AI category creator, which is used to create the categories that are used in the web application. |
| `environment.py`           | Two functions: `is_running_locally` which checks whether the app is ran locally or on Streamlit Cloud (i.e. the dedicated server for the app), and `get_gemini_api_key` which gets the API key for the Gemini API from different sources based on whether the app is ran locally or on Streamlit Cloud. |
| `back_button.py`           | Back button, which is used to navigate back to a given page. |
| `toast.py`                 | Toast notifications, which are used to display messages to the user. This specific integration allows the toast notifications to superpose everything and thus survive page changes. |
| `create_category_prompt.py`| Prompt that is used to create the categories. |
| `smiles_validator_ai_prompt.py`| Prompt that is used to validate the SMILES strings. |

Each of these functions were called throughout the `.py` files in the `pages` folder to create the web application. 

It is worth noting that the utils folder also contains a `deprecated` folder, which contains old versions of the functions that are no longer used yet are still relevant: things like DECIMER integration, or an old version of the AI validator which generated STRINGS instead of molecules.

## 3. Discussion

### 3.1 Challenges & How We Overcame Them

At the beginning of the project, our biggest challenge was figuring out how to manage the data for all the Sketchem players. The idea of storing, organizing, and tracking such a large amount of information felt overwhelming. We initially considered using **Firebase / Firestore** (Google's game-oriented database service), which is a cloud-based database. Instead, we quickly realized that Streamlit only runs one instance of every package (for everyone that accesses the website) when it is hosted on its platform Streamlit Cloud. We thus took advantage of that to create a "mock" database which would be stored in a dictionary inside of a python file of our project and would constantly be updated according to the user interactions with the website. This way, we didn't need to maintain an external database, which made things much simpler and helped us avoid extra workload.

As we worked through the project and started developing our ideas, we ran into another big challenge: molecule recognition. Initially, we wanted to use **Decimer** to convert the hand-drawn molecules into SMILES and determine whether the drawing was correct. Although Decimer was incredibly accurate when we tested it locally, we faced several issues including long waiting times (close to 7 minutes for some molecules) and issues with running the ML model on the Streamlit Cloud platform (over-allocation of resources). This pushed us to look for another solution, and that's when we found **Google's Gemini**. Since LLMs are trained on huge amounts of data, our idea was that Gemini would be an ideal contender for Decimer. After switching to Gemini, many issues such as long wait times and allocation limits were fixed. 

Despite the slight improvement with Gemini, the switch from a dedicated ML-based package to a more generic LLM led to other issues such as uneven accuracy and difficulty in adjusting prompts to fit our needs. Not all correctly drawn molecules were recognized, and occasionally, false drawings were marked as correct. Unfortunately, no solution was found for this.


Another issue arose when we decided to use Gemini for automatic category creation. As expected, Gemini's managed to create categories of molecules linked to a specific word or sentence really accurately. However, it really lagged behind when it came to finding the appropriate SMILES for the molecules in these categories. To improve this, we had the idea to connect Gemini to **PubChem**, a specialized chemistry database. This ensured all the generated SMILES were correct and didn't cause any additional issues later during the molecule recognition process.


For building the web application, we chose **Streamlit** because it's a simple tool for creating web apps. It worked well for developing the core functionality of our project without much trouble. However, Streamlit’s simplicity meant that customizing the layout and formatting was harder than we expected. The default design didn’t meet our expectations, and it often resulted in a very plain user interface. This ended up costing us more time than we had planned. Still, we were able to make the best of it by customizing the layout as much as possible using custom div IDs from the deployed HTML code of the app, and we even created different versions of the web app depending on whether the user was on a phone or a computer.

Lastly, since Streamlit did not implement a way to detect whether the player was playing on a phone or a computer, we had to find a workaround. Initally, we had planned on using st-screen-stats, but we had issues with its dependencies which we tried fixing by forking the project. Forking the project did fix the dependency issues but it caused additional issues with the installation of the newly forked project which forced us to let go of the idea. Hence, we went with using streamlit_js_eval to get the window width and determine whether the user was on a phone or a computer manually. This method worked although we did not find a way to get it to update based on changes in the window width. 


### 3.2 Educational Impact and Applications

Below is an overview of the educational impact and applications of Sketchem:

#### Chemistry Education

1. **Interactive Learning**: Makes learning chemical structures more engaging and encourages repeated practice through competition

2. **Immediate Feedback**: Helps students correct their mistakes quickly using almost-instantaneous feedback called "Hints"

3. **Close to Real Life**: Our idea behind using a canvas-based drawing platform instead of the typical bond / atom molecule drawing packages was its similarity to how one learns to draw molecules in classrooms. We strongly believe our canvas-based approach is optimized for a proper learning experience. 

4. **Teaching Tool for Educators**:
Teachers could use Sketchem in class to host real-time molecule-drawing games, quizzes, or assessments, adding fun and interaction to traditional lectures.

Sketchem also serves as a proof of concept (POC) for how multimodal AI (Gemini) can be used to recognize hand-drawn molecules and convert them to molecule names. This has applications in many fields, such as:

#### Research Applications

1. **Rapid Digitization**: Our code could be reused and perfected to create a tool like DECIMER that allows researchers to quickly digitize hand-drawn ideas, but without the cumbersome aspect of ML models nor the wait times associated with using ML-based packages like DECIMER.

2. **Quick Database Creations**: Our code could be reused and perfected to create a tool that generates a very detailed database that includes molecule names, SMILES strings and much more information, from a single user-typed prompt.

#### Future Directions

Sketchem reimagines how chemistry is learned by turning molecular structure drawing into an interactive, approachable experience. By engaging visual and motor pathways, drawing supports deeper memorization and by making the process a competitive game, Sketchem encourages practice and learning. It can be used by study groups or individuals for exam preparation or even in their free time. Since categories can be tailored to specific needs, the app is suitable for all types of chemistry courses (organic, inorganic, coordination chemistry, etc.) too. By integrating AI with multiplayer gameplay, Sketchem offers a new approach to mastering chemical structures. In addition, its user-friendly interface requires no prior training, and a built-in guide ensures that anyone can begin using the app effectively within minutes.

Regardless, there is always room to grow. Having talked about the project with our target audience: students, teachers, and chemists, we now know the future direction of the project. If it weren't for the time constraints of the project, here are a few things we would have liked to implement:

1. **Improved Recognition Accuracy**  
   Further optimization of the AI molecule recognition by fine-tuning the prompt

2. **Expanded Molecule Library**  
   Offer more curated categories (e.g., drug molecules, biomolecules) with difficulty levels, explanations, and possibly recommended game durations

3. **Advanced Game Modes**
    Reverse Mode: instead of drawing, the game would show a molecular structure and the player would need to type the correct IUPAC name; this would be perfect for practicing nomenclature and reinforcing the connection between structures and names

    Cooperative mode: would allow a user to share a canvas with their friend(s) and work together on drawing molecules

4. **Logging in**  
   Enable users to log in and save their favorite categories and progress, allowing them to see how well they are doing over time and point out which molecules are giving them trouble. Moreover, logging in would allow for a seamless use across different devices. 

5. **Multiplayer Expansion**
    Allow players to match with random opponents, letting users play competitively even if their friends are not available. 
    
    Alternatively, allow users to play their own game first and then send it to a friend for them to complete. This would maintain the competitive aspect while making it much easier to fit into every player's respective schedule (since both players would not have to be online at the same time).

6. **Custom Categories**
   Let users build fully personalized molecule sets by entering molecules manually, extending beyond the AI-generated categories to support curriculum-specific content.
   
7. **Educator Tools**  
   Let users create folders of categories and share them. This is perfect for study groups, teachers assigning homework, or students collaborating across campuses.
   

Nonetheless, our project already takes an important step in integrating AI into chemistry education. 

We can't wait for you to try it!